# Wrapping the RAG Agent into an ACP Server

We'll wrap the RAG CrewAI agent that we created in the last notebook in ACP server and then run the ACP server to activate the agent so it can be discoverable by an ACP client.

For now, we'll only make terminal 1 that later we can make another terminal 2 to connect the insurance agent with hospital.

## 4.1. Wrap the Agent in ACP  Server

We'll now take the same code you worked on in Lesson 3 and wrap it in a python file called: `crew_agent_server`.

To make the agent ACP compliant, we use the `@server.agent()` decorator to define our agent. The name is inferred from the function name, and the description is pulled from the docstring. The minimal structure needed for an ACP-compliant agent will be like this:

```python
@server.agent()
async def policy_agent(input: list[Message]) -> AsyncGenerator[RunYield, RunYieldResume]:
    "This is an agent for questions around policy coverage, it uses a RAG pattern to find answers based on policy documentation. Use it to help answer questions on coverage and waiting periods."
    # Here goes the function definition
    # ....
    task_output = ...
    yield Message(parts=[MessagePart(content=str(task_output))])
```
This configuration establishes several critical aspects of the agent:
- **Function Definition**: The core functionality that determines what the agent does;
- **Input Parameter**: The input parameter accepts a list of Message objects; 
- **Return Type**: The AsyncGenerator[RunYield, RunYieldResume] return type enables both streaming responses and the await pattern:
   - AsyncGenerator: An async generator object that can be iterated with async for and supports await operations
   - RunYield: The type of values this generator yields (sends out)
   - RunYieldResume: The type of values this generator receives when resumed (sent back in) (in the definition below, you will only use RunYield)
- **Documentation**: The docstring provides a human-readable description of the agent

Run the following cell to copy the content of the cell to the file `crew_agent_server.py` which will be saved under the folder `my_acp_project`. 

In [7]:
# Use this function to output into a file that we can run into a file using Python or UV
%%writefile ../my_acp_project/crew_agent_server.py

# convert this into an ACP server (like connecting agent to the hospital)
from collections.abc import AsyncGenerator
from acp_sdk.models import Message, MessagePart
from acp_sdk.server import RunYield, RunYieldResume, Server

# This is our last notebook on building the agent for the first time using RAG
from crewai import Crew, Task, Agent, LLM
from crewai_tools import RagTool

# create new variable here for server and assign it to our ACP server
import nest_asyncio
nest_asyncio.apply()

# making the ACP server
server = Server()
llm = LLM(model="openai/gpt-4", max_tokens=1024)

config = {
    "llm": {
        "provider": "openai",
        "config": {
            "model": "gpt-4",
        }
    },
    "embedding_model": {
        "provider": "openai",
        "config": {
            "model": "text-embedding-ada-002"
        }
    }
}
rag_tool = RagTool(config=config,  
                   chunk_size=1200,       
                   chunk_overlap=200,     
                  )
rag_tool.add("../data/gold-hospital-and-premium-extras.pdf", data_type="pdf_file")

# Define the agent to communicate to ACP server
@server.agent()
# get kick off the message (after click "Enter")
async def policy_agent(input: list[Message]) -> AsyncGenerator[RunYield, RunYieldResume]: # return our output from ACP server
    "This is an agent for questions around policy coverage, it uses a RAG pattern to find answers based on policy documentation. Use it to help answer questions on coverage and waiting periods."

    insurance_agent = Agent(
        role="Senior Insurance Coverage Assistant", 
        goal="Determine whether something is covered or not",
        backstory="You are an expert insurance agent designed to assist with coverage queries",
        verbose=True,
        allow_delegation=False,
        llm=llm,
        tools=[rag_tool], 
        max_retry_limit=5
    )

    # unpack the message and push it into CrewAI Task
    task1 = Task(
         description=input[0].parts[0].content, # it gives the server response to the user with dynamic prompt, not just specific prompt like "what is waiting period" or like mini-FAQ that has been fixed before
         # content: when we retunr the variable from description, we're able to extract all of those components. Loop and create multiple tasks
         expected_output = "A comprehensive response as to the users question",
         agent=insurance_agent
    )
    crew = Crew(agents=[insurance_agent], tasks=[task1], verbose=True)
    
    task_output = await crew.kickoff_async()
    # Then, return the task here
    yield Message(parts=[MessagePart(content=str(task_output))]) # because we are build the generator, we're gonna return the different message part from our agent 

if __name__ == "__main__":
    server.run(port=8001)

UsageError: Line magic function `%%writefile` not found.


## 4.2. Run the Insurer ACP Server

Now to activate your configured ACP agent, we need to run your agent server. The folder `my_acp_project` has been set up for you so you can run the agent server using `uv`:

- Open the terminal by running the cell below
- Type `uv run crew_agent_server.py` to run the server and activate your ACP agent.

This notebook has been installed with uv. If we want to replicate the work locally on our machine, please see below.

In [8]:
# Building IFrame so we can start it up and give it a go.
from IPython.display import IFrame
import os
url = os.environ.get('DLAI_LOCAL_URL').format(port=8888)
IFrame(f"{url}terminals/1", width=800, height=600)
# terminal 1 because eventually we're gonna run multiple ACP servers
# Note: if we want to make the hospital server, we'll later use terminal 2

We now have an agent running on port 8001 that can receive messages from others, or be called via HTTP, using the ACP protocol. 

**Note**: If you see this warning: 
`WARNING: Can not reach server, check if running on http://127.0.0.1:8333 : Request failed after 5 retries`
Then ignore it. 

If the platform is installed, it runs by default on port 8333. The ACP servers are configured to automatically connect to the platform. Since the platform isn't installed in this environment, the ACP server will generate a warning.

**Note: How to set up `my_acp_project` locally on your machine using the `uv` tool?**

- First install `uv` by checking this [link](https://docs.astral.sh/uv/getting-started/installation/).

After that, you can create `my_acp_project` in any directory of your choice, then in the terminal you can type the following commands:
- `cd my_acp_porject`
- `uv init`: to initialize the project
- `uv venv`: to create a virtual environment
- `uv add crewai crewai-tools acp-sdk load_dotenv nest-asyncio`: to define the dependencies.

Then create `crew_agent_server.py` inside the `my_acp_project`.

You can then run the server using `uv run`.  Since this code uses an OpenAI model, you would also need to specify an openAI API key in a `.env` file like this: `OPENAI_API_KEY=sk-...`. If you would like to use a local open source model using `Ollama`, please check the resource section below.

## Resources

- [How to wrap Agent](https://agentcommunicationprotocol.dev/how-to/wrap-existing-agent)
- [Configuration of ACP Agent](https://agentcommunicationprotocol.dev/core-concepts/agent-lifecycle#configuration)
- [Same code using a local open source model: `ollama_chat/qwen2.5:14b`](https://github.com/nicknochnack/ACPWalkthrough/blob/main/2.%20CrewAI%20via%20Server.py)